# Import packages

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import cv2
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers, layers

from collections import defaultdict
from tqdm import tqdm

# Setup datasets

In [ ]:
IMG_SIZE = 224
HIDDEN_LAYERS = [128, 64, 32]
USE_HIDDEN_LAYERS = False
USE_AUGUMENTATION = False
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 16
EPOCHS = 100

In [ ]:
dataset_name = "busi"
dataset_paths = [
    '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT',
    '/kaggle/input/busi-dcgan-001/breast-cancer-ultrasound-dcgan-generative'
]

In [ ]:
EXPERIMENTS_CLASSES = '__all__'

In [ ]:
IMG_SIZE = 224

data_dict = defaultdict(list)
error_count = 0
image_paths = []
images = []
labels = []
masks = []

limit = 2000

for path in dataset_paths:
    for dirpath, _, filenames in os.walk(path):
        for filename in tqdm(filenames):
            
            if 'mask' in filename or 'h5' in filename: continue
            label = dirpath.split('/')[-1]
            if len(data_dict[label]) >= limit: continue
            try:
                path = os.path.join(dirpath, filename)
                image = cv2.imread(path) # Check for no error
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                data_dict[label].append(image)
                image_paths.append(path)
                images.append(image)
                labels.append(label)
            except:
#                 print('ERROR: ', filename)
                error_count += 1

print("ErrorCount:", error_count)
print("Total Images:", len(images))

In [ ]:
{ k: len(v) for k, v in data_dict.items() }

In [ ]:
plt.figure(figsize=(9, 3))
sns.barplot(x=list(map(len, data_dict.values())), y=list(data_dict.keys()))
plt.title("\nThe Count of images in each labels\n", weight="bold")

In [ ]:
import math
plt.rcParams.update({'font.size': 14})

def visualize_datasets(images, labels, k=4, cols=4, seed=42):
    # visualize datasets

    if k > len(images): k = len(images)
    fig = plt.figure(figsize=(5 * cols, 3))
    rows = math.ceil(k / cols)

    for i in range(k):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.title.set_text(labels[i])
        plt.imshow(images[i] / 255)
        plt.colorbar()
        plt.axis('off')

    plt.show()

sample_images = [data_dict[k][1] for k in data_dict]
visualize_datasets(sample_images, list(data_dict.keys()), k=len(data_dict), cols=3)

# Data augumentation

In [ ]:
img_augmentation_layers = [
    layers.RandomRotation(factor=0.8),
    layers.RandomFlip(),
]

def img_augmentation(images, k=1):
    
    results = []
    for i in range(k):
        x = images
        for layer in img_augmentation_layers:
            x = layer(x)
        results.extend(x)
        
    return results

In [ ]:
sample_per_class = 1350
all_images = [*images]
all_labels = [*labels]

if USE_AUGUMENTATION:
    aug_dict = defaultdict(list)
    for label, _images in tqdm(data_dict.items()):
        k = int(sample_per_class / len(_images))
        if k > 0:
            aug_dict[label] = img_augmentation(_images, k)

    for label, items in aug_dict.items():
        size = len(items)
        aug_labels = [label for i in range(size)]
        all_images.extend(items)
        all_labels.extend(aug_labels)

    print(len(all_images), len(all_labels))
else:
    print("No use data augmentation.")

In [ ]:
from collections import Counter

plt.figure(figsize=(9, 3))
counter = Counter(all_labels)
sns.barplot(y=list(counter.keys()), x=list(counter.values()))
plt.title("\nThe Count of images in each labels after augmentation\n", weight="bold")

# Encode datasets

In [ ]:
images = all_images
labels = all_labels
classes = list(data_dict.keys())
num_classes = len(classes)
classes

In [ ]:
# Convert labels to numpy array
x = np.stack(images, axis=0)
y = np.array([classes.index(label) for label in labels])

x.shape, y.shape

# Split datasets

In [ ]:
# Split the data into training and remaining sets (validation + test)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
# Split the remaining data into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

[
    x_train.shape, 
    x_val.shape, 
    x_test.shape, 
    y_train.shape , 
    y_val.shape , 
    y_test.shape
]

# Helper functions

In [ ]:
def plot_acc(model_history, name):
    plt.rcParams.update({'font.size': 14})
    print('\n\n')
    epochs = len(model_history.history["accuracy"])
    plt.figure(figsize=(12,8))
    plt.plot(np.arange(0, epochs), model_history.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), model_history.history["val_accuracy"], label="val_acc")
    plt.title("Training Accuracy - {}".format(name))
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

In [ ]:
def plot_loss(model_history, name):
    plt.rcParams.update({'font.size': 14})
    print('\n\n')
    epochs = len(model_history.history["loss"])
    plt.figure(figsize=(12,8))
    plt.plot(np.arange(0, epochs), model_history.history["loss"], label="train_loss", )
    plt.plot(np.arange(0, epochs), model_history.history["val_loss"], label="val_loss")
    plt.title("Training Loss - {}".format(name))
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.rcParams.update({'font.size': 14})
    
    # plot the confusion matrix
    class_count = len(classes)
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
    plt.xticks(np.arange(class_count)+.5, classes, rotation=90)
    plt.yticks(np.arange(class_count)+.5, classes, rotation=0)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.show()

In [ ]:
def evaluate(model, x, y):
    scores = model.evaluate(x, y, verbose=1)
    return scores

In [ ]:
def predict_prob(model):
    return model.predict(x_test, batch_size=BATCH_SIZE, verbose=1)

In [ ]:
def predict(model):
    predictions = predict_prob(model)
    return np.argmax(predictions, axis=1)

In [ ]:
def calculate_metrics(y_true, y_pred):
    
    print("Visualize: y_true, y_pred top 20")
    print('Y_true', [i for i in y_true[:20]])
    print('Y_pred', [j for j in y_pred[:20]])

    # precision tp / (tp + fp)
    precision = precision_score(y_true, y_pred, average='weighted')
    print("Precision: {}".format(precision))

    # recall: tp / (tp + fn)
    recall = recall_score(y_true, y_pred, average='weighted')
    print("Recall:    {}".format(recall))

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1:        {}".format(f1))

# Setup Transfer Learning

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=25,
    restore_best_weights=True,
)

In [ ]:
def transfer_learning(model, name):
    
    best_weights_ph1 = f"{dataset_name}_{name}_ph1_weights.keras"
    
    callbacks_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath = best_weights_ph1,
        monitor = "val_accuracy",
        mode = "max",
        save_weights_only=True,
        save_best_only = True,
        verbose=1, # Logging when callback running
    )
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        validation_data=(x_val, y_val),
        validation_batch_size=BATCH_SIZE,
        epochs = EPOCHS,
        callbacks = [callbacks_checkpoint, early_stop]
    )
    
    acc_max = max(history.history["accuracy"])
    acc_min = min(history.history["accuracy"])
    print("Training Acc:", [acc_min, acc_max])
    
    val_acc_max = max(history.history["val_accuracy"])
    val_acc_min = min(history.history["val_accuracy"])
    print("Validation Acc:", [val_acc_min, val_acc_max])
    
    best_idx = np.argmax(history.history["val_accuracy"])
    print('The best val_acc result expected at epoch {} with metrics: '.format(best_idx + 1))
    
    for k, vals in history.history.items():
        print('{}: {}'.format(k, vals[best_idx]))
    
    print('\nRestoring best weights and predicting validation set.')
    model.load_weights(best_weights_ph1)
    model.save(f"{dataset_name}_{name}_ph1_model.keras")
    model.save(f"{dataset_name}_{name}_ph1_model.h5")
    
    loss, acc = evaluate(model, x_test, y_test)
    print('Transfer Learning test scores (loss, acc):', [loss, acc])
    plot_acc(history, f"\n Transfer Learning - ACC: {name} PhA.")
    plot_loss(history, f"\n Transfer Learning - LOSS: {name} PhA.")
    y_pred = predict(model)
    return history, model, val_acc_max, y_pred

# Setup fine tuning

In [ ]:
def fine_turning(model, name, acc_ph1):
    
    best_weights_ph2 = f"{dataset_name}_{name}_ph2_weights.keras"
    callbacks_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath = best_weights_ph2,
        monitor = "val_accuracy",
        mode = "max",
        save_weights_only=True,
        save_best_only = True,
        verbose=1, # Logging when callback running
    )
    
    for layer in model.layers:
        if isinstance(layer, layers.BatchNormalization) or isinstance(layer, layers.LayerNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        x_train, 
        y_train,
        batch_size=BATCH_SIZE,
        validation_data=(x_val, y_val),
        validation_batch_size=BATCH_SIZE,
        epochs = EPOCHS,
        callbacks = [callbacks_checkpoint, early_stop]
    )
    
    acc_max = max(history.history["accuracy"])
    acc_min = min(history.history["accuracy"])
    print("Training Acc:", [acc_min, acc_max])
    
    val_acc_max = max(history.history["val_accuracy"])
    val_acc_min = min(history.history["val_accuracy"])
    print("Validation Acc:", [val_acc_min, val_acc_max])
    
    best_idx = np.argmax(history.history["val_accuracy"])
    print('The best val_acc result expected at epoch {} with metrics: '.format(best_idx))
    for k, vals in history.history.items():
        print('{}: {}'.format(k, vals[best_idx]))
    
    print('Restoring best weights of Ph2 and predicting test set.')
    model.load_weights(best_weights_ph2)
    model.save(f"{dataset_name}_{name}_ph2_model.keras")
    model.save(f"{dataset_name}_{name}_ph2_model.h5")
    
    loss, acc = evaluate(model, x_test, y_test)
    print('Fine Tuning test scores (loss, acc):', [loss, acc])
    
    if val_acc_max < acc_ph1:
        print('\nPhase 2 resulted in lower accuracy than Phase 1.')
    
    plot_acc(history, f"\n Fine Tuning - ACC: {name} PhB.")
    plot_loss(history, f"\n Fine Tuning - LOSS: {name} PhB.")
    
    y_pred = predict(model)
    return history, model, val_acc_max, y_pred

# Build Model

In [ ]:
initial_models = dict(
#     EfficientNetV2B3=tf.keras.applications.efficientnet_v2.EfficientNetV2B3,
#     ResNet50V2=tf.keras.applications.resnet_v2.ResNet50V2,
#     MobileNetV2=tf.keras.applications.mobilenet_v2.MobileNetV2,
    EfficientNetB3=tf.keras.applications.EfficientNetB3,
#     ResNet50=tf.keras.applications.resnet50.ResNet50,
#     MobileNet=tf.keras.applications.mobilenet.MobileNet,
#     MLPMixer=keras_mlp.MLPMixer,
#     MLPMixerS32=keras_mlp.MLPMixerS32,
#     MLPMixerS16=keras_mlp.MLPMixerS16,
#     MLPMixerB32=keras_mlp.MLPMixerB32,
#     MLPMixerB16=keras_mlp.MLPMixerB16,
#     DenseNet169=tf.keras.applications.DenseNet169
#     Xception=tf.keras.applications.Xception
#     vit_b16=vit.vit_b16,
#     vit_b32=vit.vit_b32
)

base_model_kwargs = dict(
    include_top=False,
    weights='imagenet',
    input_shape=INPUT_SHAPE,
    pooling=None
)

# custom kwargs for each model here 
initial_models_kwargs = dict(
    EfficientNetV2B3={ **base_model_kwargs },
    ResNet50V2={ **base_model_kwargs },
    MobileNetV2={ **base_model_kwargs },
    EfficientNetB3={ **base_model_kwargs },
    ResNet50={ **base_model_kwargs },
    MobileNet={ **base_model_kwargs },
    Xception={ **base_model_kwargs },
    DenseNet169={ **base_model_kwargs },
    MLPMixer=dict(
        pretrained="imagenet",
        num_blocks=4,
        patch_size=14,
        stem_width=128,
        tokens_mlp_dim=256,
        channels_mlp_dim=512,
    ),
    MLPMixerS32=dict(
        pretrained="imagenet",
        num_classes=0
    ),
    MLPMixerS16=dict(
        pretrained="imagenet",
        num_classes=0
    ),
    MLPMixerB32=dict(
        pretrained="imagenet",
        num_classes=0
    ),
    MLPMixerB16=dict(
        pretrained="imagenet",
        num_classes=0
    ),
    vit_b16=dict(
        classes=0,
        include_top=False,
        pretrained_top=False,
        pretrained=True
    ),
    vit_b32=dict(
        classes=0,
        include_top=False,
        pretrained_top=False,
        pretrained=True
    ),
)

# Explanation setup

In [ ]:
# Adapted with some modification from https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
class GradCAM:
    
    def __init__(self, model, layerName=None):
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        
        gradModel = tf.keras.Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.output],
            name='gradcam'
        )
        
        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs)
            loss = preds[:, classIdx]
        
        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))
        
        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)
        
        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam / np.max(cam)
        cam = cv2.resize(cam, upsample_size)
        
        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1,1,3])
        
        return cam3


def overlay_gradCAM(img, cam3):
    cam3 = np.uint8(255 * cam3)
    cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
    new_img = 0.5 * cam3 + 0.5 * img
    return (new_img * 255.0 / new_img.max()).astype("uint8")

def show_gradCAMs(model, gradCAM, batch, true_label, decode_labels={}):
    
    plt.rcParams.update({'font.size': 13})
    n = len(batch)
    ncols = int(np.round(n / 2, 0))
    plt.figure(figsize=(20, ncols * 5))
    preds = model.predict(batch)
    
    for i in range(n):
        
        # Show original image
        plt.subplot(ncols, 4, 2 * i + 1)
        plt.imshow(batch[i])
        plt.title("Ground Truth: {}".format(true_label))
        plt.axis("off")
        
        # Show overlayed grad
        plt.subplot(ncols, 4, 2 * i + 2)
        pred = preds[i]
        idx = np.argmax(preds, axis=1)[i]
        res = [decode_labels[idx], float(np.max(pred))]
        
        heat_map = gradCAM.compute_heatmap(image=np.expand_dims(batch[i], axis=0), classIdx=idx, upsample_size=(IMG_SIZE, IMG_SIZE))
        overlay_img = overlay_gradCAM(batch[i], heat_map)
        plt.imshow(overlay_img)
        plt.title("Predict: {}\nConfidence: {:.4f}".format(res[0], float(res[1])))
        plt.axis("off")
        
    plt.show()

In [ ]:
def visualize_gradcam(model, title):
    print(title)
    gradCAM = GradCAM(model=model)
    for label, batch in data_dict.items():
        print(f'Label: {label}.')
        
        batch = np.stack(batch, axis=0)[:2]
        show_gradCAMs(model, gradCAM, batch, label, decode_labels=classes)

# Setup save history

In [ ]:
def save_history(history, path):
    history_df = pd.DataFrame(data=history)
    history_df.index.name = "Epoch"
    history_df.to_csv(path)
    print("Saved history completed.")
    print(history_df)

# Train

In [ ]:
last_models = dict()

for model_name, Model in initial_models.items():
    
    base_model = Model(**initial_models_kwargs[model_name])
    base_model.trainable = False
    
    model_name = "Proposed"
    x = base_model.inputs
    y = base_model.layers[-1].output
    y = layers.BatchNormalization()(y)
    y = layers.MultiHeadAttention(num_heads=8, key_dim=512, dropout=0.1)(y, y)
    y = layers.BatchNormalization()(y)
    y = layers.GlobalAveragePooling2D()(y)
    y = layers.Dense(512, activation='relu')(y)
    output = layers.Dense(num_classes, activation='softmax')(y)
    model = tf.keras.models.Model(x, output, name=model_name)
    
#     sequence_layers = [
#         base_model,
#         layers.GlobalMaxPooling2D(),
#     ]

#     if USE_HIDDEN_LAYERS:
#         for i, units in enumerate(HIDDEN_LAYERS):
#             sequence_layers.append(layers.Dense(units, activation='relu'))
#             sequence_layers.append(layers.BatchNormalization())
#             sequence_layers.append(layers.Dropout(0.2))
            
#     sequence_layers.append(layers.Dense(num_classes, activation='softmax'))

#     output = sequence_layers[0].layers[-1].output
#     for layer in sequence_layers[1:]:
#         output = layer(output)
    
#     model = tf.keras.models.Model(base_model.inputs, output, name=model_name)
    
#     model = Sequential(sequence_layers)

    print(f'\n\n ==========Start Process with model {model_name}=========')
    model.summary()

    history, model, best_acc_ph1, y_pred = transfer_learning(model, model_name)
    calculate_metrics(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cm, classes, title=f"Confusion matrix for {model_name} - Transfer Learning")
    
    save_history(history.history, f"{model_name}-{dataset_name}-transfer-results.csv")
    
    visualize_gradcam(model, "==========GRADCAM after transfer learning==========")
    
    best_acc = best_acc_ph1
    if best_acc_ph1 < 1:
        history, model, best_acc_ph2, y_pred = fine_turning(model, model_name, best_acc_ph1)
        calculate_metrics(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        plot_confusion_matrix(cm, classes, title=f"Confusion matrix for {model_name} - Fine Turnning")
        save_history(history.history, f"{model_name}-{dataset_name}-fine-tuning-results.csv")
        visualize_gradcam(model, "==========GRADCAM after fine tuning==========")
        
    else:
        print('Transfer learning have 100% accuracy so no need to do fine-turning.')

    print(f'==========End Process with model {model_name}==========\n\n')
    
    last_models[model_name] = model

# Test and Visualize Results

In [ ]:
k = 5
n = len(y_test)
sample_idx = np.random.choice(range(n), k)
x_sample = x_test[sample_idx]
y_sample = y_test[sample_idx]
y_sample

In [ ]:
plt.rcParams.update({'font.size': 16})

def format_label(label):
    return '\n'.join(label.split())

short_labels = list(map(format_label, classes))
for model_name, model in last_models.items():
    print("\n\nPrediction for {} model".format(model_name))
    y_pred = model.predict(x_sample)
    fig, ax = plt.subplots(k, 2, figsize=(30, 25))
    
    for i in range(k):
        acc = y_pred[i] * 100
        bar_colors = ['red', 'blue', 'green', 'orange', 'purple', 'gray', 'magenta']
        ax[i, 0].imshow(x_sample[i] / 255)
        ax[i, 0].axis('off')
        ax[i, 1].bar(short_labels, acc, label=short_labels, color=[bar_colors[i % len(bar_colors)] for i in range(num_classes)])
        ax[i, 1].set_ylabel('Predict')
        ax[i, 1].set_title('Ground Truth: {}'.format(classes[y_sample[i]]))

    plt.tight_layout()
    plt.show()

In [ ]:
for model_name, model in last_models.items():
    y_pred = np.argmax(model.predict(x_test), axis=1)
    print("Classification Report for {}".format(model_name))
    print(classification_report(y_test, y_pred, target_names=classes))